<a id='import'></a>
## Import libraries and data

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD
import pandas as pd

In [ ]:
#https://drive.google.com/file/d/1KBadEsrjr0OELZdon09x5bpUGRew9OIT/view?usp=sharing
#classification
#importing dataset
import gdown
import zipfile
import os

# Google Drive file ID and URL to download the file
file_id = '1KBadEsrjr0OELZdon09x5bpUGRew9OIT'
download_url = f'https://drive.google.com/uc?id={file_id}'
output_path = '/content/forestfire.zip'

# Download the file
gdown.download(download_url, output_path, quiet=False)

# Path to unzip the file
extract_path = '/content'
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipping completed.")

Downloading...
From (original): https://drive.google.com/uc?id=1KBadEsrjr0OELZdon09x5bpUGRew9OIT
From (redirected): https://drive.google.com/uc?id=1KBadEsrjr0OELZdon09x5bpUGRew9OIT&confirm=t&uuid=66dc0725-77bc-4060-ac6b-ddc88792ba8a
To: /content/forestfire.zip
100%|██████████| 383M/383M [00:07<00:00, 47.9MB/s]


Unzipping completed.


The target size used cannot be smaller than (224,224) as it will affect the calculations in the GoogleNet model.

In [ ]:

# # Paths to your datasets
# train_data_dir = '/content/content/classification_dataset/ForestFireDataset/train'
# valid_data_dir = '/content/content/classification_dataset/ForestFireDataset/valid'

# # Parameters
# batch_size = 16
# target_size = (224, 224)

# # Load the training dataset
# train = tf.keras.preprocessing.image_dataset_from_directory(
#     train_data_dir,
#     image_size=target_size,
#     batch_size=batch_size,
#     label_mode='int'  # 'int', 'categorical', or 'binary'
# )

# # Load the validation dataset
# validation = tf.keras.preprocessing.image_dataset_from_directory(
#     valid_data_dir,
#     image_size=target_size,
#     batch_size=batch_size,
#     label_mode='int'  # 'int', 'categorical', or 'binary'
# )


In [ ]:
# import glob
# fire = glob.glob('/content/dataset/ForestFireDataset/train/fire/*.*')
# nofire = glob.glob('/content/dataset/ForestFireDataset/train/nofire/*.*')

# data=[]
# labels = []

# for i in fire:
#     image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
#     image=np.array(image)
#     data.append(image)
#     labels.append(0)
# for i in nofire:
#     image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
#     image=np.array(image)
#     data.append(image)
#     labels.append(1)




In [ ]:
# data = np.array(data)
# labels = np.array(labels)

There are a total of 4934 images. Once appended, the size is (224,224,3). 3 refers to its RGB value.

In [ ]:
# data.shape

<a id='splitting'></a>
## Split the data into training, testing and validation

The data is splitted into training, testing and validation set. The ratio of training, testing and validation set is 6:2:2.

In [ ]:
# from sklearn.model_selection import train_test_split
# # Split into training set and testing set
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4,
#                                                 random_state=42)


After splitting, all of the images in the training, testing and validation sets have to be normalized (divide by 255). Normalization helps to convert an input image to values that are more familiar to the senses.

In [ ]:
# # Split into validation set and testing set
# X_valid,X_test,y_valid,y_test=train_test_split(X_test,y_test,test_size=0.5,random_state=42)

# X_train=X_train/255
# X_test=X_test/255
# X_valid=X_valid/255


In [ ]:
# print("Training set: ",X_train.shape)
# print("Validation set: ",X_valid.shape)
# print("Testing set: ",X_test.shape)



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate datasets
train_generator = train_datagen.flow_from_directory(
    '/content/content/classification_dataset/ForestFireDataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # or 'categorical' if you have multiple classes
)

valid_generator = valid_datagen.flow_from_directory(
    '/content/content/classification_dataset/ForestFireDataset/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # or 'categorical' if you have multiple classes
)

test_generator = test_datagen.flow_from_directory(
    '/content/content/classification_dataset/ForestFireDataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # or 'categorical' if you have multiple classes
)


Found 2834 images belonging to 2 classes.
Found 525 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


<a id='model-building'></a>
## Build the model

In [ ]:
# create model
def inception(x, filters):
    # 1x1
    path1 = Conv2D(filters=filters[0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)

    # 1x1->3x3
    path2 = Conv2D(filters=filters[1][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path2 = Conv2D(filters=filters[1][1], kernel_size=(3,3), strides=1, padding='same', activation='relu')(path2)

    # 1x1->5x5
    path3 = Conv2D(filters=filters[2][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path3 = Conv2D(filters=filters[2][1], kernel_size=(5,5), strides=1, padding='same', activation='relu')(path3)

    # 3x3->1x1
    path4 = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    path4 = Conv2D(filters=filters[3], kernel_size=(1,1), strides=1, padding='same', activation='relu')(path4)

    return Concatenate(axis=-1)([path1,path2,path3,path4])


def auxiliary(x, name=None):
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu',kernel_regularizer=regularizers.l2(0.0001))(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(units=CLASS_NUM, activation='softmax', name=name)(layer)
    return layer


def googlenet():
    layer_in = Input(shape=IMAGE_SHAPE)

    # stage-1
    layer = Conv2D(filters=64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(layer_in)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    layer = BatchNormalization()(layer)

    # stage-2
    layer = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Conv2D(filters=192, kernel_size=(3,3), strides=1, padding='same', activation='relu')(layer)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)

    # stage-3
    layer = inception(layer, [ 64,  (96,128), (16,32), 32]) #3a
    layer = inception(layer, [128, (128,192), (32,96), 64]) #3b
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)

    # stage-4
    layer = inception(layer, [192,  (96,208),  (16,48),  64]) #4a
    aux1  = auxiliary(layer, name='aux1')
    layer = inception(layer, [160, (112,224),  (24,64),  64]) #4b
    layer = inception(layer, [128, (128,256),  (24,64),  64]) #4c
    layer = inception(layer, [112, (144,288),  (32,64),  64]) #4d
    aux2  = auxiliary(layer, name='aux2')
    layer = inception(layer, [256, (160,320), (32,128), 128]) #4e
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)

    # stage-5
    layer = inception(layer, [256, (160,320), (32,128), 128]) #5a
    layer = inception(layer, [384, (192,384), (48,128), 128]) #5b
    layer = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(layer)

    # stage-6
    layer = Flatten()(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(units=256, activation='linear',kernel_regularizer=regularizers.l2(0.0001))(layer)
    main = Dense(units=CLASS_NUM, activation='softmax', name='main')(layer)

    model = Model(inputs=layer_in, outputs=[main, aux1, aux2])

    return model

In [ ]:
CLASS_NUM = 1
BATCH_SIZE = 16
# EPOCH_STEPS = int(X_train.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 3)
MODEL_NAME = 'googlenet_fire.h5'

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
!mkdir googlenet_result

In [ ]:
from tensorflow.keras import metrics
from keras.callbacks import EarlyStopping,ModelCheckpoint

checkpoint_cb =ModelCheckpoint("/content/googlenet_result/googlenet_model.keras", save_best_only=True)

# train model
model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

# optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy',
#         metrics.Precision(name='precision'),
#         metrics.Recall(name='recall'),
#         tf.keras.metrics.AUC(name='auc'),
#         tf.keras.metrics.AUC(name='pr_auc', curve='PR'),])
model.compile(
    optimizer='adam',
    loss={'main': 'binary_crossentropy', 'aux1': 'binary_crossentropy', 'aux2': 'binary_crossentropy'},
    metrics={'main': ['accuracy', metrics.Precision(name='precision'),
        metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR')],
             'aux1': ['accuracy', metrics.Precision(name='precision'),
        metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR')],
             'aux2': ['accuracy', metrics.Precision(name='precision'),
        metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR')],}
)
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)


history = model.fit(train_generator,epochs=200,validation_data=valid_generator,callbacks=[checkpoint_cb])

# model.save(MODEL_NAME)



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 112, 112, 64)   │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 56, 56, 64)     │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 56, 56, 64)     │            256 │ max_pooling2d[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 56, 56, 64)     │          4,160 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 56, 56, 192)    │        110,784 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 56, 56, 192)    │            768 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 28, 28, 192)    │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 28, 28, 96)     │         18,528 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 28, 28, 16)     │          3,088 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 28, 28, 192)    │              0 │ max_pooling2d_1[0][0]  │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 28, 28, 64)     │         12,352 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 28, 28, 128)    │        110,720 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 28, 28, 32)     │         12,832 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 28, 28, 32)     │          6,176 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 28, 28, 256)    │              0 │ conv2d_3[0][0],        │
│                           │                        │                │ conv2d_5[0][0],        │
│                           │                        │                │ conv2d_7[0][0],        │
│                      

 Total params: 7,420,211 (28.31 MB)

 Trainable params: 7,419,699 (28.30 MB)

 Non-trainable params: 512 (2.00 KB)

...

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (18, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 944ms/step - loss: 0.5789 - main_accuracy: 0.3021 - main_auc: 0.5000 - main_pr_auc: 0.3021 - main_precision: 0.3021 - main_recall: 1.0000

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (13, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


89/89 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 0.5766 - main_accuracy: 0.3021 - main_auc: 0.5000 - main_pr_auc: 0.3021 - main_precision: 0.3021 - main_recall: 1.0000 - val_loss: 0.8051 - val_main_accuracy: 0.0952 - val_main_auc: 0.5000 - val_main_pr_auc: 0.0952 - val_main_precision: 0.0952 - val_main_recall: 1.0000
Epoch 2/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 120s 971ms/step - loss: 0.2076 - main_accuracy: 0.3086 - main_auc: 0.5000 - main_pr_auc: 0.3086 - main_precision: 0.3086 - main_recall: 1.0000 - val_loss: 0.2648 - val_main_accuracy: 0.0952 - val_main_auc: 0.5000 - val_main_pr_auc: 0.0952 - val_main_precision: 0.0952 - val_main_recall: 1.0000
Epoch 3/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 141s 952ms/step - loss: 0.1391 - main_accuracy: 0.3130 - main_auc: 0.5000 - main_pr_auc: 0.3130 - main_precision: 0.3130 - main_recall: 1.0000 - val_loss: 0.3251 - val_main_accuracy: 0.0952 - val_main_auc: 0.5000 - val_main_pr_auc: 0.0952 - val_main_precision: 0.0952 - val_main_recall: 1.0000
Epoch 4/200
89/8

KeyboardInterrupt: 

In [ ]:
hist_=pd.DataFrame(history.history)
hist_.index.name = 'epochs'
hist_
hist_.to_csv('googlenet_result/training_history_googlenet.csv')

In [ ]:
!zip -r googlenet_result.zip /content/googlenet_result
from google.colab import files
files.download("googlenet_result.zip")

<a id='model-evaluation'></a>
## Model Evaluation

In [ ]:
score = model.evaluate(test_generator)
print('Score:', score[4])


In [ ]:
# plt.plot(history.history['main_accuracy'])
# plt.plot(history.history['val_main_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()


In [ ]:
# plt.plot(history.history['main_loss'])
# plt.plot(history.history['val_main_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()